# Scripts du cours Spark SQL

Attention, si vous utilisez la version comprenant Jupyter (utilisation de docker-compose-jupyter.yml pour les Mac par exemple) , utilisez la configuration suivante (cellule désactivée) de Spark à la place de la suivante.



In [1]:
### Configuration Mac : utilisation de notebook Jupyter

from pyspark import SparkContext, SparkConf

conf = SparkConf() \
    .setAppName('SparkApp') \
    .setMaster('spark://spark:7077') \
    .set("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4") # utilisé pour le stockage 

sc = SparkContext(conf=conf)

from pyspark.sql import SQLContext
# Créer un SQLContext pour les opérations SQL
sql_context = SQLContext(sc)

minio_ip_address = "minio"

:: loading settings :: url = jar:file:/opt/conda/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-83eeac6e-3d5f-4ace-8bc4-902c773877ad;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 258ms :: artifacts dl 9ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------

## DataFrames vs Views

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg

# Create DataFrame
data = [("Brooke", 20), ("Brooke", 25), ("Denny", 31), ("Jules", 30), ("TD", 35)]
df = sql_context.createDataFrame(data, ["name", "age"])
# Calculate average age by name using DataFrame API
avgDF = df.groupBy("name").agg(avg("age"))
avgDF.show()# Create a temporary view to use SQL
df.createOrReplaceTempView("people")
# Calculate average age by name using SQL
sqlDF = sql_context.sql("""
SELECT name, avg(age) as avg_age
FROM people
GROUP BY name
""")
sqlDF.show()

25/02/25 08:29:52 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/02/25 08:30:07 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/02/25 08:30:22 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/02/25 08:30:37 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/02/25 08:30:52 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/02/25 08:31:07 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure th

KeyboardInterrupt: 

## Colonnes et expressions sur les colonnes

In [ ]:
from pyspark.sql.functions import col, expr
print(df.columns)

print(df["name"])

print(df.select(col("age") + 1).first())

print(df.select(df.age + 1).first())

print(df.select(expr("age + 1")).first())

## Lignes

In [ ]:
from pyspark.sql import Row

first_row = df.first()
print(first_row)  # Output: Row(name='Brooke', age=20)

tail_rows = df.tail(2)
print(tail_rows)  # Output: [Row(name='Jules', age=30), Row(name='TD', age=35)]

row = Row("Brooke", 20)
print(row)  # Output: Row(Brooke=20)

print(row[0])  # Output: Brooke
print(row[1])  # Output: 20

# Convert sequence to DataFrame
seq_df = sql_context.createDataFrame([("Brooke", 20)], ["name", "age"])
seq_df.show()

## Projection et sélection

In [ ]:
# Apply transformations
result = (
    df.select("name")             # Projection: Select only the "name" column
      .where("age <= 30")         # Selection: Filter rows where "age" is less than or equal to 30
      .distinct()                 # Eliminate duplicates
).show()


## Renommage, ajout et suppression de colonnes

In [ ]:
df.withColumnRenamed("name", "nom").show(1)

from pyspark.sql.functions import floor, col
df.withColumn("dizaine", floor(col("age") / 10))  \
    .drop("age") \
    .show(1)

## Agrégats

In [ ]:
from pyspark.sql.functions import max
result = (
    df.groupBy("name")       # Group by "name"
      .agg(max("age").alias("max_age"))  # Calculate max "age"
).show()
from pyspark.sql.functions import min, max, avg
df.select(min("age"), max("age"), avg("age")).show()

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql import Row

# Define schema with StructType and StructField
schema = StructType([
    StructField("nom", StringType(), nullable=False),
    StructField("age", IntegerType(), nullable=False)
])

# Create data
data = [Row("Brooke", 20)]

# Create DataFrame
df = sql_context.createDataFrame(data, schema)

# Show DataFrame
df.show()

## Files

In [ ]:
# Create a sample CSV file
import os

csv_content = """name,age
Brooke,20
Jules,30
Cameron,35"""

with open("sample.csv", "w") as file:
    file.write(csv_content)

print("Sample CSV file created!")

In [ ]:


sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", f"http://{minio_ip_address}:9000")
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", "root")
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "password")
sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")

from minio import Minio
client_minio = Minio(
    f"{minio_ip_address}:9000",
    access_key="root",
    secret_key="password",
    secure=False
)

# Création du bucket tp6
if client_minio.bucket_exists("tp6") == False:
    client_minio.make_bucket("tp6")
client_minio.fput_object("tp6", "sample.csv", "sample.csv")    

In [ ]:
# Read CSV file
df = (
    sql_context.read
    .format("csv")                  # Specify format as CSV
    .option("inferSchema", "true")  # Infer the schema automatically
    .option("header", "true")       # The file contains a header
    .option("mode", "FAILFAST")     # Fail if there are any errors
    .option("nullValue", "")        # Treat empty strings as null values
    .load("s3a://tp6/sample.csv")                 # Path to the CSV file
)

# Show the DataFrame
df.show()

In [ ]:
from pyspark.sql.functions import col

# Example: Add a new column and filter rows
df_transformed = (
    df.withColumn("age_in_5_years", col("age") + 5)  # Add 5 to the age
      .filter(col("age") < 30)                      # Keep rows where age is less than 30
)

# Show the transformed DataFrame
df_transformed.show()

In [ ]:
(df_transformed.coalesce(1).write           # execute on one task
    .format("csv")                          # output format 
    .option("header", "true")               # header present
    .mode("overwrite")                      # overwrite if exist
    .save("s3a://tp6/transformed_sample"))  # directory name

Vous pouvez aller vérifier sur [Minio](http://localhost:19001/browser/tp6/) (pour rappel utilisateur et mot de passe : root / password ) dans le bucket TP6 que le fichier a bien été généré (dans le dossier `transformed_sample`). 